# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [14]:
# imports
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display

In [15]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [16]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [17]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

messages = [
    {
        "role": "system",
        "content": "You are an assistant that will answer technical questions for programming. When giving try to give in an explanation and and example. \
        Explaining the parts of the example if any. Respond in markdown format"
    },
    {
        "role": "user",
        "content": question
    }
]


In [20]:
# Get gpt-4o-mini to answer, with streaming

def stream_response(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

def call_open_ai_stream(prompt):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=prompt,
        stream=True
    )
    
    stream_response(stream)


call_open_ai_stream(messages)

Certainly! The code snippet you've provided utilizes a combination of a generator expression with the `yield from` statement and a set comprehension. Let's break it down step-by-step.

### Breakdown of the Code

1. **Set Comprehension**:
   python
   {book.get("author") for book in books if book.get("author")}
   
   - This part of the code creates a set containing the authors of the books.
   - **`books`**: It is assumed to be a list (or any iterable) of book dictionaries.
   - **`book.get("author")`**: This method retrieves the value associated with the key `"author"` in each book dictionary. If the key does not exist, `get` returns `None` instead of raising a KeyError.
   - **`if book.get("author")`**: This condition filters out books that do not have an author. Only books with a non-`None` value for the author will be included in the resulting set.
   - The result is a set of unique authors since sets do not allow duplicate values.

2. **`yield from`**:
   python
   yield from ...
   
   - The `yield from` expression is used inside a generator function. It delegatesthe yielding of values from the iterable (in this case, the set of authors) back to the caller.
   - This allows the generator to yield each element of the set one at a time, maintaining the generator's state between yields.

### Example Code

Here’s an example to illustrate how this code works:

python
def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Example list of books
books = [
    {"title": "Book A", "author": "Author 1"},
    {"title": "Book B", "author": "Author 2"},
    {"title": "Book C"},                    # No author
    {"title": "Book D", "author": "Author 1"},  # Duplicate author
    {"title": "Book E", "author": None},    # None author
]

# Using the generator
for author in get_unique_authors(books):
    print(author)


### Explanation of the Example

- **Books List**: The `books` variable is a list of dictionaries, where each dictionary represents a book, having a title and possibly an author.
- **`get_unique_authors` function**: This generator function processes the `books` iterable, uses the set comprehension to filter and extract authors, then yields each unique author found.
- **Output**: When the generator is called, it will print:
  
  Author 1
  Author 2
  
  - Only the authors who are present in the books and are unique are output, while books without authors are ignored.

### Summary

In summary, the code snippet builds a set of unique authors from a list of books and uses `yield from` to output each author one by one through a generator. This approach is memory efficient and clean, especially useful when dealing with large datasets or when you want to process results on-the-fly.

In [21]:
# Get Llama 3.2 to answer

def call_ollama_stream(prompt):
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=prompt,
        stream=True
    )

    stream_response(stream)

call_ollama_stream(messages)
    

**Unpacking a Nested Generator**
=====================================

The provided code snippet is using generator expression and `yield from` to extract the authors of books from a list. Here's a breakdown:

python
yield from {book.get("author") for book in books if book.get("author")}


*   `{...}` : This syntax creates an empty dictionary comprehension, which might seem unusual at first. However, it serves a purpose similar to `dict comprehension`.

However there is not enough information about the code structure and libraries used. So Below I'll give 2 possible structure.

### Structure 1

python
import collections.abc

def get_books(books):
    for book in books:
        yield from {
            key: value 
            for key, value in book.items()
            if key != 'author' or (key == 'author' and value)
        }

def find_authors():
    authors = {}
    for book in books:
        author = next(book.values())
        if not author:
            continue
        if author not in authors.values():
            yield author 

# Example usage
books = [
    {"title": "Book1", "author": "John"},
    {"title": "Book2", "author": ""},
    {"title": "-3-Book3", "author": "Alessandro"}
]
books = [book for book in books if book] # drop the empty books

authors = set(list(find_authors()))
print(authors) 

In this code `get_books` is function that yield each line of a JSON. We are also filtering out some lines (as requested).

python
    def find_authors():
        authors = {}
        for book in books:
            author = next(book.values())
            if not author:
                continue
            if author not in authors.values():
                yield author 

`find_authors()` function filters duplicate value and also skipping on None.